In [1]:
%display latex

<h1><p style="color:#16A085"> Álgebra Computacional </p style></h1>
<h2> Índice </h2>

- 1. Algoritmo de Euclides
    - 1.1 Algoritmo de Euclides para un Dominio Euclídeo
    - 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo
    - 1.3 Ejemplos en $\mathbb{Z}$
    - 1.4 Ejemplos en $\mathbb{Q}[x]$
- 2. Teorema chino del resto
    - 2.1 Ejemplos en $\mathbb{Z}$
- 3. MCD en DFU
    - 3.1 Ejemplos en $\mathbb{Z}[x]$
- 4. Inverso de un elemento en un cuerpo finito
    - 4.1 Ejemplos en $\mathbb{Z}_p$
- 5. Test de irreducibilidad de un polinomio en $\mathbb{F}_q[x]$
    - 5.1 Ejemplos en $\mathbb{F}(7^2)[x]$
- 6. Logaritmo discreto en cuerpos $\mathbb{F}_q[x]/(f(x))$
    - 6.1 Ejemplos en $\mathbb{F}(3^6)[x]/(bx + 2b)$
- 7. Algoritmo de factorización de un polinomio en cuerpo finito
    - 7.1 Equal degree-splitting
    - 7.2 Equal degree-factorization
    - 7.3 Algoritmo de factorización de un polinomio en cuerpo finito
    - 7.4 Ejemplos en $\mathbb{F}(3)[x]$
- 8. Algoritmo de factorización de Berlekamp en cuerpo finito
    - 8.1 Ejemplos en $\mathbb{F}(7)[x]$
- 9. Algoritmos de factorización en $\mathbb{Z}[x]$
    - 9.1
- 10. Algoritmo de primalidad de *AKS*
    - 10.1

# 1. Algoritmo de Euclides

# 1.1 Algoritmo de Euclides para un Dominio Euclídeo

In [2]:
def euclides(a, b, mod):
    if (b == 0):
        return a
    else:
        return euclides (b, (mod(a, b)), mod)

## 1.2 Algoritmo de Euclides extendido para un Dominio Euclídeo

In [3]:
def extendedEuclides(a, b, div_mod):
    if (b == 0):
        return a, 1, 0 
    else:
        q, r = div_mod(a, b)
        gcd, d, e = extendedEuclides(b, r, div_mod)
        return gcd, e, (d -q * e)

## 1.3 Ejemplos en $\mathbb{Z}$

In [4]:
def mod_int(a, b): return a % b
def div_mod_int(a, b): return a // b, a % b

print(euclides(55, 22, mod_int))
print(extendedEuclides(55, 22, div_mod_int))

11
(11, 1, -2)


## 1.4 Ejemplos en $\mathbb{Q}[x]$

In [5]:
def div_mod_pol(a, b): return a.quo_rem(b);
def mod_pol(a, b): _, r =  div_mod_pol(a, b); return r
    
R.<x> = QQ[]

a = (x^2 +2*x + 1)
b =  (x^2 - 1)

print(euclides(a, b, mod_pol))
print(extendedEuclides(a, b, div_mod_pol))

c = (x+2/3)*(x-3/2)*(x^2+1)
d = (x+2/3)*(x+2/3)*(x^2+1)
print(euclides(c, d, mod_pol)).monic()

2*x + 2
(2*x + 2, 1, -1)
x^3 + 2/3*x^2 + x + 2/3


# 2. Teorema chino del resto

In [6]:
def chineseRemainder(coprimes, elems, div_mod, rem):
    m = 1
    for mi in coprimes:
        m = m * mi
    result = 0;
    for i in range(len(coprimes)):
        n = m / coprimes[i]
        _, a, _ = extendedEuclides(n, coprimes[i], div_mod)
        c = rem((a * elems[i]), coprimes[i])
        result = c * n + result
    return result

## 2.1 Ejemplos en $\mathbb{Z}$

In [7]:
def rem(a, b):
    r = abs(a) % abs(b)
    return r if a >= 0 else -r
chineseRemainder([3,5,7],[4,5,33], div_mod_int, rem)

40

# 3. MCD en DFU

In [8]:
def gcdDFU(a, b):
    if a.degree() < b.degree():
        return mcdDFU(b,a)
    while b != 0:
        q,r = (b.leading_coefficient()**(a.degree() - b.degree() + 1) * a).quo_rem(b)
        a = b
        b = r
    return a

## 3.1 Ejemplos en $\mathbb{Z}[x]$

In [9]:
R.<x> = ZZ[]
a = (x + 1)**2
b = (x + 1)*(x - 1)
gcdDFU(a, b).monic()

x + 1

# 4. Inverso de un elemento en un cuerpo finito

In [10]:
def inverseFiniteField(a, field, div_mod):
    _, _, b = extendedEuclides(field.cardinality(), a, div_mod)
    return field(b)

## 4.1 Ejemplos en $\mathbb{Z}_p$

In [11]:
Z7 = Integers(7)
a = 3
print(inverseFiniteField(a, Z7, div_mod_int))

Z31 = Integers(31)
b = 14
print(inverseFiniteField(b, Z31, div_mod_int))

Z61 = Integers(61)
c = 27
print(inverseFiniteField(c, Z61, div_mod_int))

5
20
52


# 5. Test de irreducibilidad de un polinomio en $\mathbb{F}_q[x]$

In [12]:
def fast_exp_mod(b,e,m):
#Devuelve b^e mod m.
    r = 1
    b = b%m
    while e>0:
        if e%2 == 1:
            r = (r*b)%m
        e = e//2
        b  = (b*b)%m
    return r 

def irreducibility_test(f,q):
    h = (x) % f
    l = f.degree()
    for k in range(1, l // 2 + 1):
        h = fast_exp_mod(h,q,f)
        if gcd(h-x, f) != 1: return False
    return True

## 5.1 Ejemplos en $\mathbb{F}(7^2)[x]$

In [13]:
order = 49
R.<x>=PolynomialRing(GF(order, 'z'))
f = x^21+x^2+1
print "El polinomio ", f, " es irreducible en F49[x]?: ", irreducibility_test(f,order), "."
g = R.random_element(6)
print "El polinomio ", g, " es irreducible en F49[x]?: ", irreducibility_test(g,order), "."

El polinomio  x^21 + x^2 + 1  es irreducible en F49[x]?:  False .
El polinomio  x^6 + (6*z + 4)*x^5 + 2*z*x^4 + (z + 6)*x^3 + (3*z + 6)*x^2 + 5*x + 6*z + 2  es irreducible en F49[x]?:  True .


# 6. Logaritmo discreto en cuerpos $\mathbb{F}_q[x]/(f(x))$

In [14]:
def baby_giant(v,g,ordG):
    m = ceil(sqrt(ordG))

    #  Generamos baby = { g^(j): 0 <= j < n }
    baby = [g^0]
    for i in  xrange(1,m): baby.append( baby[i-1]*g )
    
    # Generamos giant = { h*g^(-i): 0 <= i < w }
    giant = [v]
    inv_gm = g^(-m)
    for i in  xrange(1,m): giant.append( giant[i-1]*inv_gm )
     
    # Tomamos los elementos que están simultáneamente en baby y giant
    # Nos quedamos con el que tenga menor índice en giant.
    # Para ese índice, calculamos x = j*m + i
    # Si la intersección es vacía, no existe el logaritmo
    intersection = set(baby).intersection(set(giant))
    if len(intersection) == 0: 
        print ("No existe logaritmo discreto de %s en base " + \
        "%s") % (v,g)
    else:
        b = len(giant)
        for inters in intersection:
            if giant.index(inters) < b:
                b = giant.index(inters)
                a = baby.index(inters)
                x = a+b*m
        
        #print  'x = a + b*m= (%s) + (%s)*(%d) = (%s)' % (a,b,m,x)  
        return x

## 6.1 Ejemplos en $\mathbb{F}(3^6)[x]/(bx + 2b)$

In [15]:
K.<b> = GF(3^6)
R.<x> = PolynomialRing(K)
f = b*x + 2*b # polinomio de K[x]
I = R.ideal([f]) # ideal generado por bx + 2b
Q = R.quotient_ring(I) # K[x]/f(x)

Ax = Q(b*x^3 + x + b)
ordA = order_from_multiple(Ax,Q.order(),operation='*',check=False)

Bx = Ax^245
Cx = Q(x^2 + b)

print(baby_giant(Bx,Ax,ordA))
print(baby_giant(Cx,Ax,ordA))

21
No existe logaritmo discreto de b + 1 en base 2*b + 1
None


# 7. Algoritmo de factorización de un polinomio en cuerpo finito

## 7.1 Equal degree-splitting

In [16]:
def eq_degree_splitting(f, q, d):
    R = f.parent()
    a = R.random_element(0, f.degree() - 1) # 0 < deg a < deg f

    if (a.degree() < 1): return "failure"

    g1 = gcd(a,f)
    if (g1 != 1): return g1
    
    b = fast_exp_mod(a, (q^d - 1) // 2, f)
    
    g2 = gcd(b - 1, f)  
    if (g2 != 1 and g2 != f): return g2
    else: return "failure"

## 7.2 Equal degree-factorization

In [17]:
def eq_degree_factorization(f,q,d):

    if (f.degree() == d): return [f]
    while(True):
        g = eq_degree_splitting(f, q, d)
        if (g != "failure"): break
    return eq_degree_factorization(g, q, d) + eq_degree_factorization(f // g, q, d)

## 7.3 Algoritmo de factorización de un polinomio en cuerpo finito

In [18]:
def poly_fact_finite_field(f,q):
    R=f.parent()
    h=x
    v=f/f.leading_coefficient() 
    i=0
    U=[]
    while (True): 
        i=i+1
        h=fast_exp_mod(h,q,f)
        g=gcd(h-x,v)
        if (g!=1):
            G=eq_degree_factorization(g,q,i) #Computar los factores mónicos irreducibles de g

            for gj in G:
                e=0
                while((v%gj)==0): # Calcular multiplicidades
                    
                    v=v//gj 
                    e=e+1
                U=U+[(gj,e)]

        if (v==1): break 
    
    return U

## 7.4 Ejemplos en $\mathbb{F}(3)[x]$

In [23]:
R.<x>=PolynomialRing(GF(3))
f=x^4+x^3+x-1
f2=x^8+x^7-x^6+x^5-x^3-x^2-x
print (f.factor())
print (poly_fact_finite_field(f,3))
#print (poly_fact_finite_field(f2,3))

(x^2 + 1) * (x^2 + x + 2)


KeyboardInterrupt: 

# 8. Algoritmo de factorización de Berlekamp en cuerpo finito


In [76]:
def berlekamp(f,q):

    R=f.base_ring()
    RX=f.parent()
    n=f.degree()
    pol=1
    aux=fast_exp_mod(x,q,f)
    Q=matrix(R,n,n)

    #Rellenamos Q de forma que Q[i,j]:= coeficiente de (x^j) en (x^(qi) mod f)

    for i in range(n):
        for j in range(n): Q[i,j]=0

    #Rellenamos la primera fila "a mano"
    Q[0,0]=1

    #En cada iteración calculamos (x^(qi) mod f) y rellenamos la fila i-ésima de Q
    for i in range(1,n):
        pol=(pol*aux)%f
        for j in range(pol.degree()+1): 
            Q[i,j]=pol[j]

    #Calculamos Q-I
    for i in range(n): Q[i,i]=Q[i,i]-1

    Q.echelonize() #Efectúa transformación por el metodo de eliminacion Gaussiana
    K=Q.kernel()
    B=K.basis_matrix() #Base de la Berlekamp algebra
    r=K.dimension()

    if (r==1): return f

    C=random_vector(R,r)

    a=RX([0]) #Inicializamos la variable, porque da eror si no 
    
    for i in range(r): #Calculamos a = c1*b1 + c2*b2 + ...cr*br
        a=a+C[i]*RX(B[i].list()) 


    g1=gcd(a,f)

    if g1!=1 and g1!=f: 
        return g1

    b=fast_exp_mod(a, ((q-1)//2), f)
    g2=gcd(b-1,f)

    if g2!=1 and g2!=f:
        return g2
    else: 
        return "failure"


## 8.1 Ejemplos en $\mathbb{F}(7)[x]$

In [80]:
def berlekamp_test(f,q):
    print "f = ", f
    g=berlekamp(f,q)
    if g!="failure": 
        print "g = ",g
        h=f/g
        print "h = ", h
        print "g*h = ", g*h
    else: 
        print "No se ha encontrado un factor g para el polinomio dado"

In [81]:
R.<x>=PolynomialRing(GF(7))
f= 6*x^9 + 2*x^8 + x^7 + 5*x^6 + 5*x^5 + 6*x^3 + 2*x^2 + 3
berlekamp_test(f,7)

f =  6*x^9 + 2*x^8 + x^7 + 5*x^6 + 5*x^5 + 6*x^3 + 2*x^2 + 3
g =  x + 3
h =  6*x^8 + 5*x^7 + 5*x^5 + 4*x^4 + 2*x^3 + 2*x + 1
g*h =  6*x^9 + 2*x^8 + x^7 + 5*x^6 + 5*x^5 + 6*x^3 + 2*x^2 + 3


In [88]:
f=R.random_element((4,10))
berlekamp_test(f,7)

f =  x^6 + 6*x^5 + 2*x^4 + 6*x^2 + 2*x
g =  x
h =  x^5 + 6*x^4 + 2*x^3 + 6*x + 2
g*h =  x^6 + 6*x^5 + 2*x^4 + 6*x^2 + 2*x


# 9. Algoritmos de factorización en $\mathbb{Z}[x]$

# 10. Algoritmo de primalidad de *AKS*

In [24]:
def AKS(n):
    # comprobamos que no sea potencia de la forma a^b
    for b in xrange(2,log(n,2)):
        a = n^(1/b)
        if floor(a)==a: return 'composite'

    # Obtenemos r tal que O_r(n) > log_2(n)^2
    r=2; nextR=True
    m=pari(log(n,2)^2).floor() 
    M=max(3, pari(log(n,2)^5).ceil())
    while nextR and r<M:
        nextR=False
        k=1
        while not nextR and k<= m:
            mod = power_mod(n,k,r)
            nextR = (mod==1) or (mod==0)
            k+=1
        r+=1
    r-=1

    # si 1 < gcd(a,n) < n para a en (1,r] -> compuesto
    for a in xrange(r,1,-1):
        mcd = gcd(a,n)
        if 1 < mcd  and mcd < n: return 'composite'

    # si r>=n -> primo
    if(r>=n): return 'prime'

    # para a en [1,N] si (x+a)^n == x^n+ a en (Z_n[x] / (x^r - 1)) -> comp
    N=pari(sqrt(euler_phi(r))*log(n,2)).floor()
    for a in xrange(1,N+1):
        R.<x>=PolynomialRing(Integers(n))
        S=R.quotient((x^r)-1)
        c=S((x^n + a))
        d=(x+a)^n

        if (c!=d): return 'composite'
    return 'prime'

## 10.1 Ejemplos

In [34]:
print(AKS(3))
print(AKS(111))
print(AKS(557))

prime
composite
prime
